# Import Dependencies

Begin by importing the necessary libraries.

In [ ]:
# System & OS
import os
import glob
import warnings
warnings.filterwarnings('ignore')

# Data Storage
from google.colab import drive

# Data Analysis
import pandas as pd
from tqdm.notebook import tqdm

# Mount Storage

For simplicity, we keep the data in a Google Drive folder, and simply mount the Google Drive to our Colab instance, as if it were a local file system.

In [ ]:
# Mount Google Drive to Colab Instance
drive.mount('/content/drive')
# Change directory to where the data are stored (old: %cd '/content/drive/MyDrive/Research/Ongoing/Protostellar Luminosity/Data')
%cd '/content/drive/MyDrive/Colab Notebooks/Data'

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1EKq5q8_K4j5NQpcWKFDhxA77zKtxtbug/Data


# Data Analysis

In [ ]:
with open('master_file.csv', 'w') as master_file:
  master_file.write('Model number, Class, Mass ratio, Timestep, Inclination, Wavelength (cm), Flux (), L_int (Lsun)\n')
master_file.close()

In [ ]:
model_num_list = ['01', '02', '10', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '30', '31', '32', '33']
timesteps = [str(_) for _ in range(2, 96)]
inclinations = ['05', '15', '25', '35', '45', '55', '65', '75', '85']

In [ ]:
# Loop over every model
for i, j in enumerate(tqdm(model_num_list)):
  # Read in the internal luminosity from the model's luminosities file
  df = pd.read_table(f'models/luminosities_model{j}.tbl', 
                      skiprows=1, 
                      delim_whitespace=True, 
                      names=['Time (Myr)',
                            'Time-t0 (yr)',
                            'L_EtoD (Lsun)',
                            'L_DM (Lsun)',
                            'L_DtoS (Lsun)',
                            'L_EtoS (Lsun)',
                            'L_DR (Lsun)',
                            'L_PHOT (Lsun)',
                            'L_INT (Lsun)'])
  lint = df['L_INT (Lsun)']
  # Read in the mass of the star, disk, and envelope from the model's parameters file
  df = pd.read_table(f'models/model_parameters_model{j}.tbl', 
                      skiprows=1, 
                      delim_whitespace=True, 
                      names=['Time (Myr)',
                            'Time-t0 (yr)',
                            'Mstar (Msun)',
                            'Lstar (Lsun)',
                            'Rstar (Rsun)',
                            'Tstar (K)',
                            'Rdisk_in (AU)',
                            'Rdisk_out (AU)',
                            'Mdisk (Msun)',
                            'Renv_in (AU)',
                            'Renv_out (AU)',
                            'Menv (Msun)',
                            'Omega_0 (1/s)',
                            'c_s (cm/s)'])
  mStar, mDisk, mEnv = df['Mstar (Msun)'], df['Mdisk (Msun)'], df['Menv (Msun)']
  # Open master file to record features
  with open('master_file.csv', 'a') as master_file:
    # Initialize inner progress bar
    with tqdm(total=len(timesteps) * len(inclinations), leave=False) as pbar:
      # Loop over every spectrum file
      for a, b in enumerate(timesteps):  
        for c, d in enumerate(inclinations):
          # Check that the spectrum file exists 
          if os.path.isfile(f'models/run_evolpapiii_newdisk_model{j}/RESULTS/spectrum_{b}_inc{d}.dat'): 
            # Read in the frequency and flux from the spectrum file
            df = pd.read_table(f'models/run_evolpapiii_newdisk_model{j}/RESULTS/spectrum_{b}_inc{d}.dat', 
                              skiprows=2, 
                              delim_whitespace=True, 
                              names=['Frequency',
                                     'Flux'])
            frequency, flux = df['Frequency'], df['Flux']
            # Adjust flux data to be consistent with Dunham's previous work
            flux = [val * frequency[idx] * (1.0 / 140.0)**2 for idx, val in enumerate(flux)]
            # Compute the internal vs final mass ratio
            mass_ratio = (mStar[a + 1] + mDisk[a + 1]) / (mStar[a + 1] + mDisk[a + 1] + mEnv[a + 1])
            # We define class 1 to be when the mass_ratio >= 0.5
            class_split = 1 if mass_ratio >= 0.5 else 0
            # Loop over all frequencies
            for e, f in enumerate(frequency):
              # Write our results to the master file
              master_file.write(f'{j}, {class_split}, {mass_ratio}, {b}, {d}, {2.99792458e10 / f}, {flux[e]}, {lint[a + 1]}\n')
          # Update inner progress bar
          pbar.update(1)
  master_file.close()

NameError: ignored